In [ ]:
import requests
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from selenium import webdriver
from datetime import datetime
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from konlpy.tag import Kkma, Hannanum, Komoran, Mecab
kkma = Kkma()
mecab = Mecab()

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

from sklearn.preprocessing import normalize
import re
from gensim.summarization.summarizer import summarize

In [ ]:
# 네이버 크롤링 코드
article_df = pd.DataFrame(columns=['date','title','contents','press','good','well','sad','mad','want'])
scan_day = pd.datetime(2021,1,10) # 시작 일자 설정
page_count = True
page = 1

for a in range(0, 10) : 

    search_day = str(scan_day.year) + str(scan_day.month).zfill(2) + str(scan_day.day).zfill(2)
    print(search_day)
    while page_count :
        main_url = 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date={}&page={}'.format(search_day, page)
        headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
        main_res = requests.get(main_url, headers = headers)
        main_soup = BeautifulSoup(main_res.text, 'html.parser')
        path = '#main_content > div.list_body.newsflash_body > ul > li'
        page_path1 = '#main_content > div.paging > a'
        page_path2 = '#main_content > div.paging > strong'
        main_select = main_soup.select(path)
        page_select = main_soup.select(page_path1)
        present = main_soup.select(page_path2)
        if page_select[-1].text != '다음' :
            if 20 != len(main_select) :
                page_count = False
            elif int(present[0].text) == int(page_select[-1].text)+1  :
                page_count = False

        page += 1            
        print(main_url)
        try : 
            for i in range(0, len(main_select)) :
                news_cont_url = main_select[i].select('a')[0]['href']

                cont_res = requests.get(news_cont_url, headers = headers)
                cont_soup = BeautifulSoup(cont_res.text, 'html.parser')
                cont_select_head = cont_soup.select('#main_content > div.article_header')
                cont_select_body = cont_soup.select('#articleBodyContents')

                target_text_without_child_tags = [bs_object for bs_object in cont_select_body[0]
                    if isinstance(bs_object, NavigableString)]
                text = "".join(target_text_without_child_tags[8:-3])

                date = cont_select_head[0].select('span.t11')[0].text
                title = cont_select_head[0].select('div.article_info #articleTitle')[0].text
                contents = text
                press = cont_select_head[0].select('a')[0]['href']
                good = cont_select_head[0].select('span.u_likeit_list_count._count')[0].text
                well = cont_select_head[0].select('span.u_likeit_list_count._count')[1].text
                sad = cont_select_head[0].select('span.u_likeit_list_count._count')[2].text
                mad = cont_select_head[0].select('span.u_likeit_list_count._count')[3].text
                want = cont_select_head[0].select('span.u_likeit_list_count._count')[4].text

                data = {'date':date, 'title':title, 'contents':contents, 'press':press, 'good':good,
                       'well':well, 'sad':sad, 'mad':mad, 'want':want}
                article_df = article_df.append(data, ignore_index=True)
        except :
            continue

    scan_day = scan_day - pd.Timedelta("1day")
    page_count = True
    page = 1

# article_df.to_csv('article_info.csv', encoding='utf-8-sig')

In [ ]:
# contents 요약
summa = []
for contents in df['contents'] :
    try : 
        sentence = kkma.sentences(contents)        
        if len(sentence) > 10 :
            summary = ''
            for row in summarize(contents).split('\n') :
                summary += row
            if summary.strip() == '' :
                summa.append(np.nan)
            else : 
                summa.append(summary)
        else :
            summa.append(np.nan)
    except :
        summa.append(np.nan)

df['summary'] = summa
df